In [ ]:
!git clone https://github.com/tardigrade-dot/SenseVoice


In [ ]:
import os
os.chdir('SenseVoice')

In [ ]:
%pip install -r requirements.txt

In [ ]:
device = "mps" # cuda cpu

In [ ]:
#下载模型
from modelscope import snapshot_download
snapshot_download('FunAudioLLM/SenseVoiceSmall', local_dir='pretrained_models/SenseVoiceSmall')

In [ ]:
from funasr import AutoModel
from funasr.utils.postprocess_utils import rich_transcription_postprocess

model_dir = "pretrained_models/SenseVoiceSmall"

src_file = 'data-forcedaligner/data.wav'

model = AutoModel(
    model=model_dir,
    # vad_model="fsmn-vad",
    # vad_kwargs={"max_single_segment_time": 30_000},
    device=device,
    # hub="hf",
    disable_update=True,
)
res = model.generate(
        input=src_file,
        cache={},
        language="auto",  # "zn", "en", "yue", "ja", "ko", "nospeech"
        use_itn=True,
        batch_size_s=30,
        # output_timestamp=True,
        merge_vad=True,
        # ban_emo_unk=True,
        merge_length_s=20,
    )
text = res[0]["text"]
print(text)
#文本输出预期为:政府对各地的请愿一概不予理睬，议会以种种理由退回请愿书，借口大多是请愿书文具粗陋，符合议会体力。政府的这种态度自然为更激烈的激进派别提供了活动的土壤。
#修正为文本:政府对各地的请愿一概不予理睬，议会以种种理由退回请愿书，借口大多是请愿书文具粗陋，不合议会体例。政府的这种态度自然为更激烈的激进派别提供了活动的土壤。
#并在下一个步骤使用到修正的文本来对齐

In [ ]:
import re
import os
import sys
from funasr import AutoModel
from funasr.utils.postprocess_utils import rich_transcription_postprocess

model_dir = "pretrained_models/SenseVoiceSmall"

src_file = 'data-forcedaligner/data.wav'
target_txt = "政府对各地的请愿一概不予理睬，议会以种种理由退回请愿书，借口大多是请愿书文具粗陋，不合议会体例。政府的这种态度自然为更激烈的激进派别提供了活动的土壤。"
model = AutoModel(
    model=model_dir,
    # vad_model="fsmn-vad",
    # vad_kwargs={"max_single_segment_time": 30_000},
    device=device,
    # hub="hf",
    disable_update=True,
    trust_remote_code=True,
    remote_code="./model2.py",
)
res = model.generate(
        input=src_file,
        cache={},
        language="auto",  # "zn", "en", "yue", "ja", "ko", "nospeech"
        use_itn=True,
        batch_size_s=30,
        # output_timestamp=True,
        merge_vad=True,
        # ban_emo_unk=True,
        merge_length_s=20,
        output_timestamp=True,
        target_txt=target_txt,
    )
text = res[0]
print(text["words"])
print(text["timestamp"])